<a href="https://colab.research.google.com/github/semhoun/omnius/blob/main/nb/FrenchDPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### Definitions


In [7]:
import os

if "COLAB_" in "".join(os.environ.keys()):
  from google.colab import userdata
  hf_token = userdata.get('HuggingFaceToken')
else:
  hf_token = os.environ["HUGGINGFACETOKEN"]

hf_username = "nsemhoun"
debug = True
save_hf = False
version = '3B'

if version == '9B':
  source_model = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"
  quantization_method = ["q8_0", "f16", "q4_k_m", "q4_0"]
  model_name = "Claire-9B-v0.1.1"
else:
  source_model = "HuggingFaceTB/SmolLM3-3B"
  quantization_method = ["q8_0", "f16"]
  model_name = "Claire-3B-v0.1.1"

### Installation

In [3]:
%%capture
import os, re
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

if "COLAB_" in "".join(os.environ.keys()):
    print("Install colab version")
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --upgrade --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps --upgrade unsloth
    !pip install transformers==4.55.4
    !pip install --no-deps trl==0.22.2
elif os.environ["USER"] == "unsloth":
    pass
else:
    !pip install --upgrade unsloth-zoo
    !pip install --upgrade unsloth
    !pip install transformers==4.55.4
    !pip install --no-deps trl==0.22.2

### Unsloth

In [4]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = source_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
    load_in_8bit = False,
    token = hf_token,
)

==((====))==  Unsloth 2025.9.5: Fast Smollm3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HuggingFaceTB/SmolLM3-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [
french-orca-dpo-pairs-revised](https://huggingface.co/datasets/jpacifico/french-orca-dpo-pairs-revised).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [9]:
dpo_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(sample):
    #instruction = "You are an AI assistant. You will be given a task. You must generate a correct answer."
    instruction = sample["system"]
    input       = sample["prompt"]
    accepted    = sample["chosen"]
    rejected    = sample["rejected"]

    sample["prompt"]   = dpo_prompt.format(instruction, input, "")
    sample["chosen"]   = accepted + EOS_TOKEN
    sample["rejected"] = rejected + EOS_TOKEN
    return sample
pass

from datasets import load_dataset
dataset = load_dataset("jpacifico/french-orca-dpo-pairs-revised", split = "train")
dataset = dataset.rename_column("question", "prompt")
dataset = dataset.map(formatting_prompts_func,)

README.md:   0%|          | 0.00/676 [00:00<?, ?B/s]

french_orca_rlhf_revised.jsonl:   0%|          | 0.00/44.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12670 [00:00<?, ? examples/s]

Map:   0%|          | 0/12670 [00:00<?, ? examples/s]

In [10]:
import pprint
row = dataset[1]
print('NB Row: ' + str(len(dataset)))
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["prompt"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('REJECTED: ' + '=' * 50)
pprint.pprint(row["rejected"])

NB Row: 12670
INSTRUCTION: ==================================================
('Below is an instruction that describes a task, paired with an input that '
 'provides further context. Write a response that appropriately completes the '
 'request.\n'
 '\n'
 '### Instruction:\n'
 'Vous êtes un assistant IA. Une tâche vous sera confiée. Vous devez générer '
 'une réponse détaillée et longue.\n'
 '\n'
 '### Input:\n'
 'Le restaurant Midsummer House propose une cuisine chinoise de gamme de prix '
 'modérée, noté 3 sur 5 par les clients, situé près de All Bar One.\n'
 '\n'
 '### Response:\n')
ACCEPTED: ==================================================
('Midsummer House est un restaurant chinois au prix modéré, noté 3/5, situé à '
 'proximité du All Bar One.<|im_end|>')
REJECTED: ==================================================
('Bien sûr! Voici une phrase qui décrit toutes les données que vous avez '
 'fournies\xa0:\n'
 '\n'
 '"Midsummer House est un restaurant chinois à prix modéré avec u

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [11]:
lora_rank = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


In [12]:
# Enable reward modelling stats
from unsloth import PatchDPOTrainer
PatchDPOTrainer()
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60 if debug else -1, # None for full run
        num_train_epochs = 0 if debug else 1,
        learning_rate = 2e-4, # Lower for slower but more precise fine-tuning. Try values like 1e-4, 5e-5, or 2e-5
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # remove to activate WandDB
    ),
    beta = 0.1,
    train_dataset = dataset,
    tokenizer = tokenizer,
    max_length = 2048,
    max_prompt_length = 1024,
)

Extracting prompt in train dataset (num_proc=16):   0%|          | 0/12670 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=16):   0%|          | 0/12670 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=16):   0%|          | 0/12670 [00:00<?, ? examples/s]

In [13]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
11.486 GB of memory reserved.


In [14]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,670 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 30,228,480 of 3,105,327,104 (0.97% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-280.850006,-411.362457,1.413870,2.388655,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-512.863586,-454.491760,1.987292,2.390002,No Log,No Log,No Log
3,0.629800,0.028012,-0.106910,0.875000,0.134922,-484.898773,-545.338501,1.508854,2.353466,No Log,No Log,No Log
4,0.627700,-0.076867,-0.215063,0.875000,0.138195,-436.885986,-562.198730,1.420187,1.348574,No Log,No Log,No Log
5,0.535400,-0.104960,-0.472089,0.875000,0.367130,-235.419708,-389.310364,2.046487,2.520015,No Log,No Log,No Log
6,0.478400,-0.089218,-0.642653,0.875000,0.553436,-189.139465,-260.286499,2.397312,2.090974,No Log,No Log,No Log
7,0.206400,-0.307171,-2.021754,1.000000,1.714583,-211.210526,-489.342834,1.388021,1.815471,No Log,No Log,No Log
8,0.278700,-0.444589,-2.902884,0.875000,2.458295,-237.902954,-369.767029,0.073028,1.590481,No Log,No Log,No Log
9,0.401900,-1.064779,-2.279279,0.875000,1.214501,-241.017380,-288.603943,2.139220,1.716831,No Log,No Log,No Log
10,0.038000,-0.536490,-4.914193,1.000000,4.377703,-160.688232,-382.089111,1.773280,1.809939,No Log,No Log,No Log


TrainOutput(global_step=60, training_loss=0.12126292839714854, metrics={'train_runtime': 589.5756, 'train_samples_per_second': 0.814, 'train_steps_per_second': 0.102, 'total_flos': 0.0, 'train_loss': 0.12126292839714854, 'epoch': 0.037884767166535126})

In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

Peak reserved memory = 16.258 GB.
Peak reserved memory for training = 4.772 GB.
Peak reserved memory % of max memory = 73.363 %.
Peak reserved memory for training % of max memory = 21.533 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [16]:
# formatting_prompts_func = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    dpo_prompt.format(
        "Continue la séquence de fibonnaci", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
        "", # rejected - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id = tokenizer.eos_token_id)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue la séquence de fibonnaci\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21\n\n### Explanation:\nThe Fibonacci sequence starts with 1 and 1, and each subsequent number is the sum of the previous two. Given the sequence 1, 1, 2, 3, 5, 8, the next numbers are calculated as follows: 5 + ']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    dpo_prompt.format(
        "Continue la séquence de fibonnaci", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
        "", # rejected - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

13, 21

### Explanation:
The sequence of Fibonacci numbers starts with 1 and 1, and each subsequent number is the sum of the previous two. The input sequence follows this pattern. The next two numbers in this sequence are 13 and 21, which complete the sequence. 

### Input:
1, 1, 2, 3, 5, 8, 13

### Response:
21, 34

### Explanation:
The Fibonacci sequence is built by adding the last two numbers to find the next number. In the given sequence, after 13, adding 8 and 13 gives 


<a name="Save"></a>
### Saving

In [18]:
#@title Create HF repository
from huggingface_hub import HfApi

if save_hf:
  hf_api = HfApi(token=hf_token)
  hf_api.create_repo(repo_id = hf_username + "/" + model_name, repo_type = "model", private = True, exist_ok = True)
  hf_api.create_repo(repo_id = hf_username + "/" + model_name + "-GGUF", repo_type = "model", private = True, exist_ok = True)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
model.save_pretrained_merged(model_name, tokenizer)
if save_hf: model.push_to_hub_merged(hf_username + "/" + model_name, token = hf_token)

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...
Successfully copied all 2 files from cache to Claire-3B-v0.1.0.


Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
model.save_pretrained_gguf(model_name, tokenizer, quantization_method = quantization_method)

if save_hf:
  for quant in quantization_method:
    hf_api.upload_file(
      path_or_fileobj=model_name + "." + quant.upper() + ".gguf",
      path_in_repo=model_name + "-" + quant.upper() + ".gguf",
      repo_id=hf_username + "/" + model_name + "-GGUF"
    )

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 18.44 out of 46.98 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/32 [00:00<?, ?it/s]
We will save to Disk and not RAM now.
 31%|███▏      | 10/32 [00:03<00:07,  2.86it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 11.63 GiB of which 55.69 MiB is free. Process 14383 has 104.00 MiB memory in use. Process 3045081 has 11.46 GiB memory in use. Of the allocated memory 11.15 GiB is allocated by PyTorch, and 148.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
